# About python imports

No imports are specified as in previous homeworks, please insert the necessary python code yourself.

-----
# Exercise 1 (3 points)
Write a program implementing Rayleigh quotient iteration for computing an eigenvalue and corresponding eigenvector of a matrix. Test your program on the matrix 
$$
  A = \begin{bmatrix} 6 & 2 & 1 \\ 2& 3 & 1 \\ 1 & 1 & 1  \end{bmatrix}
$$
using a random starting vector. Let the program create output that shows the convergence behavior.

In [ ]:
import numpy as np
def ray(A, x0, iterations):
  xold = x0
  I = np.identity(np.shape(A)[0])
  numerator = np.matmul(np.matmul(np.transpose(xold), A), xold)
  denomenator = np.matmul(np.transpose(xold), xold)
  sigma = numerator/denomenator
  print(sigma)
  for i in range(iterations):
    ynew = np.linalg.solve(A-(sigma * I), xold)
    xnew = ynew/np.linalg.norm(ynew, ord=np.inf)
    xold = xnew
    numerator = np.matmul(np.matmul(np.transpose(xnew), A), xnew)
    denomenator = np.matmul(np.transpose(xnew), xnew)
    sigma = numerator/denomenator
    print(sigma)
    print(xnew)
  

  
  return sigma, xnew




In [ ]:
A = np.array([[6,2,1], [2,3,1], [1,1,1]])
eigval, eigvec = ray2(A, np.transpose(np.array((1,1,1))), 10)
np.linalg.eig(A)

6.0
7.153846153846153
[1.  0.4 0.1]
[ 1.   0.4 -0.8]
7.287908163680387
[1.         0.5266558  0.24572227]
[1.         0.52979661 0.2716541 ]
7.287992138960401
[1.         0.5229001  0.24219175]
[1.         0.52290002 0.24219142]
7.287992138960422
[1.         0.52290017 0.24219181]
[1.         0.52290017 0.24219181]
7.287992138960422
[1.         0.52290017 0.24219181]
[1.         0.52290017 0.24219181]
7.287992138960422
[1.         0.52290017 0.24219181]
[1.         0.52290017 0.24219181]
7.287992138960422
[1.         0.52290017 0.24219181]
[1.         0.52290017 0.24219181]
7.287992138960422
[1.         0.52290017 0.24219181]
[1.         0.52290017 0.24219181]
7.287992138960422
[1.         0.52290017 0.24219181]
[1.         0.52290017 0.24219181]
7.287992138960422
[1.         0.52290017 0.24219181]
[1.         0.52290017 0.24219181]


(array([7.28799214, 2.13307448, 0.57893339]),
 array([[ 0.86643225,  0.49742503, -0.0431682 ],
        [ 0.45305757, -0.8195891 , -0.35073145],
        [ 0.20984279, -0.28432735,  0.9354806 ]]))

In [ ]:
print(np.dot(A, eigvec))
np.dot(eigval, eigvec)

[7.28799214 3.81089231 1.76509197]


array([7.28799214, 3.81089231, 1.76509197])

# Automatic differentiation using JAX

In applications of rootfinding, computing the derivative is often a problematic step. For example, the function for which zeros are sought might be given by a complicated computer program. 

Automatic differentiation is a set of techniques to evaluate the derivative of a function specified by a computer program. See the [wikipedia page](https://en.wikipedia.org/wiki/Automatic_differentiation) on this topic.

[JAX](https://github.com/google/jax) is a software package that implements automatic differentiation as well as other functionality. [The documentation is here](https://jax.readthedocs.io/en/latest/). The idea of this exercise is to use JAX for obtaining derivative functions.

To use JAX, there are two options:
- install JAX. The installation of JAX is described on the Github page, see https://github.com/google/jax#installation.  **It appears that installation under Windows is not supported. According to the internet, one may use the Windows Subsystem for Linux, but I haven't tested this.**
- run your python notebook on the google colab environment. The google colab environment is at https://colab.research.google.com. In the google colab environment, the JAX package is available. 

There is some material online about JAX, see for example
https://medium.com/swlh/solving-optimization-problems-with-jax-98376508bd4f
(LATEX-pdf version here
https://github.com/mazy1998/Solving-Optimization-Problems-with-JAX/blob/master/Opitimization_with_jax.pdf)
or 
https://www.kaggle.com/aakashnain/tf-jax-tutorials-part1.

The result is that for many functions, the derivative can be automatically computed. We will show this for a vector valued function $\mathbb{R}^2 \to \mathbb{R}^2$. 

The first step is to import some functions from the package JAX. Notice that JAX has its own version of numpy. Here we import it as `jnp`.

In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import jacfwd, jacrev 

# depending on the application, more imports are needed

JAX implements forward and reverse mode automatic differentiation. The commands are `jacfwd` and `jacrev` (jac stands for Jacobian). The [wikipedia page on automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation) briefly introduces forward and reverse mode automatic differentiation. Here we just mention that forward accumulation is more efficient than reverse accumulation for functions $f : \mathbb{R}^n \to \mathbb{R}^m$ with $m \gg n$ as only $n$ sweeps are necessary, compared to $m$ sweeps for reverse accumulation and that reverse accumulation is more efficient than forward accumulation for functions $f : \mathbb{R}^n \to \mathbb{R}^m$ with $m \ll n$ as only $n$ sweeps are necessary.

In the following cell a simple function is defined and differentiated. Note that JAX has its own array type, `jax.numpy.array` (because of the line `import jax.numpy as jnp` we write this as `jnp.array`), that output is in 32 bits floating point format in a different array type `DeviceArray` (JAX has a preference for the 32 bits floating point format and you are allowed to use it in this exercise). 

In [ ]:
def circle(x): return x[0]**2 + x[1]**2
J = jacfwd(circle)
J(jnp.array([1.0 ,2.0]))

DeviceArray([2., 4.], dtype=float32)

It is allowed to take derivatives of derivatives, so that a second derivative matrix can be obtained.

In [ ]:
def hessian(f): return jacfwd(jacrev(f))
H = hessian(circle)
myMatrix = H(jnp. array ([1.0 ,2.0]) )
print(myMatrix)

[[2. 0.]
 [0. 2.]]


Although this is not the standard `numpy.ndarray` format, it appears however that this format can be used in linear algebra operations such as solve.

In [ ]:
myVector = jnp.array([0.5, 2.0])

import scipy.linalg as la
la.solve(myMatrix, myVector)

array([0.25, 1.  ], dtype=float32)

It is easy to define vector valued functions, by returning an `jax.numpy.array` object. 

When using functions such as $\sin$ and $\cos$ and $\exp$ one must be careful. One must use the functions `jax.numpy.sin`, `jax.numpy.cos`, etc. (and not `math.sin` etc.). We define a test function $f(x_1,x_2) = [x_1 \exp(x_2), x_1+x_2]$ using $\exp$ and show that it can be differentiated. Note that the derivative matrix is $\begin{bmatrix} \exp(x_2) & x_1 \exp(x_2) \\ 1 & 1 \end{bmatrix}$.

In [ ]:
def f(x):
    return jnp.array([x[0] * jnp.exp(x[1]), x[0] + x[1]])

print("values:")
print(f(jnp.array([2.0,1.0])))

Df = jacfwd(f)
print("jacobian matrix:")
print(Df(jnp.array([2.0,1.0])))


values:
[5.4365635 3.       ]
jacobian matrix:
[[2.7182817 5.4365635]
 [1.        1.       ]]


# Exercise 2 (rootfinding with automatic differentiation, 3 points)

## (a)
Create a Python function to apply Newton's method in multiple dimensions. Create a stopping criterion, such that your method automatically stops when one of the following conditions is satisfied: (i) the size of the function is below a specified tolerance; (ii) the difference in two subsequent iterates $\mathbf{x}_k$ is below a specified tolerance; (iii) the number of iterations reaches a specified limit.

In [ ]:
def fx(x):
  assert len(x) == 2
  x0 = x[0]
  x1 = x[1]
  return jnp.array([x0 + 2*1 - 2, x0**2+4*x1**2 - 4])


def newton(f, x0, xtol, ytol, ittol):
  jacob = jacfwd(f)
  i = 0
  while(True):
    xnew = x0 - np.linalg.solve(jacob(x0), f(x0))
    i += 1
    if ((i > ittol) or (np.linalg.norm(x0-xnew, 2) < xtol) or (np.linalg.norm(f(xnew), 2) < ytol)):
      print(i)
      break
    x0 = xnew
  return xnew


In [ ]:
import numpy as np
newton(fx, jnp.array([1.0,1.0]), 0.000001, 0.000001, 10)

1.0077822
1.0625
0.11805558
0.055748463
0.006920457
0.00019168854
2.3961067e-05
0.0
4


DeviceArray([0., 1.], dtype=float32)

## (b)
Solve Computer Exercise 5.19 using Newton's method and automatic differentiation. (N.B. Do not choose the starting point equal to a solution.)

In [ ]:
def func(input):
  gamma = 5
  delta = 1
  x = input[0]
  y = input[1]
  return jnp.array([gamma*x*y - x*(1+y), -x*y+(delta-y)*(x+y)])
newton(func, jnp.array([np.random.random(),np.random.random()]), 0.000001, 0.000001, 69)

0.33565947
0.10828187
0.19897598
0.08073628
0.07238997
0.009813663
0.0103874225
0.00021230002
0.0002442131
1.3328004e-07
5


DeviceArray([-0.37500012,  0.2500001 ], dtype=float32)

In [ ]:
print(func([-0.37500012,  0.2500001]))
print(func([0,1]))

[-1.5000005e-07  6.5000016e-08]
[0 0]


Explanation using $\LaTeX$ here

# Exercise 3 (3 points)

## (a)

Consider the system 
$$\begin{aligned}(x_1+3)(x_2^3-7) + 18 = {}& 0 \\
\sin(x_2 e^{x_1} -1) = {}& 0 .
\end{aligned}$$
Solve this system using Newton's method with starting 
point $\mathbf{x}_0 = [ 0.5 \;\; 1.4 ] ^T$.


In [ ]:
def func(x):
  x1 = x[0]
  x2 = x[1]
  return jnp.array([(x1+3)*(x2**3-7) + 18, jnp.sin(x2*jnp.exp(x1) - 1)])



In [ ]:
newton(func, jnp.array([0.5, 1.4]), 0.000001, 0.000001, 69)

1.3766923
4.7527747
1.1680925
2.344731
0.4455688
1.0683753
0.09424418
0.030336076
0.0035265852
2.614464e-06
1.3399125e-06
1.9073486e-06
1.7429757e-07
0.0
7


DeviceArray([1.1176766e-08, 1.0000000e+00], dtype=float32)

In [ ]:
func([0,1])

DeviceArray([0., 0.], dtype=float32)

## (b)
Write a program based on Broyden's method to solve the same system with the same starting point.

In [ ]:
def broyden(f, x0, xtol, ytol, ittol):
  jacob = jacfwd(f)
  jacobguess = jacob(x0)
  i = 0
  while(True):
    s = np.linalg.solve(jacobguess, -f(x0))
    xnew = x0 + s
    
    # update jacobian
    y = f(xnew) - f(x0)
    numerator = y - (jacob(s) * jnp.transpose(s))
    denomenator = jnp.dot(jnp.transpose(s),s)
    jacobguess = jacobguess + numerator/denomenator

    # tolerance checking
    i += 1
    if ((i > ittol) or (np.linalg.norm(x0-xnew, 2) < xtol) or (np.linalg.norm(f(xnew), 2) < ytol)):
      print(i)
      break
    # update x
    x0 = xnew


  return xnew

In [ ]:
broyden(func, jnp.array([0.5, 1.4]), 0.0001, 0.00001, 69)

26


DeviceArray([-0.54651445, -1.0429845 ], dtype=float32)

In [ ]:
func([-0.54651445, -1.0429845 ])

DeviceArray([-1.9580644, -0.9994537], dtype=float32)

## (c)
Compare the convergence rates of the two methods by computing the error at each iteration and appropriately analysing these errors, given that the exact solution is $\mathbf{x}^* = [ 0 \;\; 1 ]^T$.

In [ ]:
# your answer here

Explanation using $\LaTeX$ here